In [1]:
import time
import hmac
from requests import Request


In [2]:
ts = int(time.time() * 1000)
request = Request('GET', 'https://ftx.com/api/markets')
prepared = request.prepare()
signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
signature = hmac.new('6Z8DbNljOhTk57q3VkrZB6-prxS0V55QvlFuiAYE'.encode(), signature_payload, 'sha256').hexdigest()

request.headers['FTX-KEY'] = '-t42j_9B6GImumwftN2Eogk0kcFcU6UsBoIYGMW6'
request.headers['FTX-SIGN'] = signature
request.headers['FTX-TS'] = str(ts)

In [4]:

print(json.dumps(request.json(), indent=2))

TypeError: 'NoneType' object is not callable

In [6]:
r = request.json

In [7]:
request.status_code

AttributeError: 'Request' object has no attribute 'status_code'

In [14]:
import pandas as pd
import requests

markets = requests.get('https://ftx.com/api/markets').json()
df = pd.DataFrame(markets['result'])
df.set_index('name', inplace = True)
df.T

name,1INCH-PERP,1INCH-0924,1INCH/USD,AAPL-0924,AAPL/USD,AAVE-PERP,AAVE-0924,AAVE/USD,AAVE/USDT,ABNB-0924,...,XRPHALF/USD,XRPHEDGE/USD,XTZBEAR/USD,XTZBEAR/USDT,XTZBULL/USD,XTZBULL/USDT,XTZHALF/USD,XTZHEDGE/USD,ZECBEAR/USD,ZECBULL/USD
enabled,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
postOnly,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
priceIncrement,0.0001,0.0001,0.0001,0.01,0.01,0.01,0.01,0.01,0.01,0.005,...,1.0,0.005,0.0,0.0,0.0001,0.0001,0.5,0.01,0.0005,0.0005
sizeIncrement,1.0,1.0,1.0,0.01,0.01,0.01,0.01,0.01,0.01,0.025,...,0.00001,0.001,10000.0,10000.0,0.1,0.1,0.00001,0.001,0.1,0.1
minProvideSize,1.0,1.0,1.0,0.01,0.01,0.01,0.01,0.01,0.01,0.025,...,0.00001,0.001,10000.0,10000.0,0.1,0.1,0.00001,0.001,0.1,0.1
last,3.1234,3.1529,3.1185,154.69,154.83,394.34,399.59,394.11,394.48,157.12,...,19557.0,46.635,0.000019,0.00002,0.4455,0.4449,9885.0,105.64,1.151,1.149
bid,3.1223,3.1534,3.1185,154.58,154.55,394.34,398.89,394.04,393.55,157.135,...,19741.0,45.55,0.000019,0.000019,0.445,0.4438,10022.0,102.29,1.164,1.1355
ask,3.1234,3.1617,3.1217,154.69,154.6,394.49,398.9,394.11,394.12,157.225,...,19786.0,45.67,0.00002,0.00002,0.4461,0.4473,10067.5,103.17,1.1785,1.146
price,3.1234,3.1534,3.1185,154.69,154.6,394.34,398.9,394.11,394.12,157.135,...,19741.0,45.67,0.000019,0.00002,0.4455,0.4449,10022.0,103.17,1.164,1.146
type,future,future,spot,future,spot,future,future,spot,spot,future,...,spot,spot,spot,spot,spot,spot,spot,spot,spot,spot


In [11]:
print(df.index)

Index(['1INCH-PERP', '1INCH-0924', 'AAPL-0924', 'AAVE-PERP', 'AAVE-0924',
       'ABNB-0924', 'ACB-0924', 'ADA-PERP', 'ADA-0924', 'ALCX-PERP',
       ...
       'XRP-0924', 'XTZ-PERP', 'XTZ-0924', 'YFI-PERP', 'YFI-0924', 'YFII-PERP',
       'ZEC-PERP', 'ZIL-PERP', 'ZM-0924', 'ZRX-PERP'],
      dtype='object', name='name', length=256)


In [15]:
df.to_csv('mu.csv')

In [16]:
markets = requests.get('https://ftx.com/api/markets').json()
df_markets = pd.DataFrame(markets['result'])
df_markets.set_index('name', inplace = True)
df_markets['symbol'] = df_markets.index

In [29]:
db_columns = ['last','bid','ask','price','change1h','change24h','changeBod','volumeUsd24h']
df_market_db = pd.DataFrame(df_markets.loc['BTC/USD',db_columns]).T

In [24]:

import os
import psycopg2
import requests
import json
import datetime
from pytz import timezone

In [23]:
!pip install psycopg2-binary

     |████████████████████████████████| 3.4 MB 5.9 MB/s eta 0:00:01


In [25]:
host="rdsnske.c7sczatjnphy.ap-northeast-1.rds.amazonaws.com"
dbname="bot"
user="rdsuser"
password="rds%5678"
insert_base = "insert into ftx_market_btc values('{basetime}','{symbol}','{last}','{bid}','{ask}','{price}','{change1h}','{change24h}','{changeBod}','{volumeUsd24h}');"

In [33]:
a = "geogho{f}"
b = a.format(f=df_market_db["last"].values)

In [47]:
# データをDBに登録
with psycopg2.connect(host=host, dbname=dbname, user=user, password=password) as conn:
    with conn.cursor() as cur:
        basetime = datetime.datetime.now()
        basetime = basetime.astimezone(timezone('Asia/Tokyo'))
        basetime = datetime.datetime.strftime(basetime, '%Y-%m-%dT%H:%M:00')
        insert_sql = insert_base.format(basetime=basetime
                                        ,symbol='BTC/USD'
                                        ,last=df_market_db["last"].values[0]
                                        ,bid=df_market_db["bid"].values[0]
                                        ,ask=df_market_db["ask"].values[0]
                                        ,price=df_market_db["price"].values[0]
                                        ,change1h=df_market_db["change1h"].values[0]
                                        ,change24h=df_market_db["change24h"].values[0]
                                        ,changeBod=df_market_db["changeBod"].values[0]
                                        ,volumeUsd24h=df_market_db["volumeUsd24h"].values[0])
        cur.execute(insert_sql)
    conn.commit()

In [45]:
f=df_market_db["last"].values[0]